Motivation: In this notebook, I give an example of how to use my implementation of fast Kronecker inference for GPs with arbitrary likelihoods. See Flaxman et al (2015) "Fast Kronecker Inference in Gaussian Processes with non-Gaussian Likelihoods" for reference.

In [1]:
from kronecker import KroneckerSolver
import optimizers as opt
from likelihoods import PoissonLike
import simulator as sim
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from IPython.display import display
init_notebook_mode(connected=True)
from edward import rbf
import GPy
import numpy as np

## Simulate some data

First, let's simulate some data on a 2D grid. Given a set of points $\{x_i\}_{i=1}^N$, we generate data via the following model:

$$f \sim ~\mathcal{GP}(\mu(x), K(x, x))$$

$$ y(x_i) \sim ~ \text{Poisson}(f(x_i))$$


For some intuition, here's what the $x_i$'s look like:

In [2]:
X = sim.sim_X(N_dim = 30)

iplot([go.Scatter(x = X[:,0], y = X[:,1], mode = 'markers', marker=dict(size = 2,))])

Now we draw function values f from a GP with an RBF Kernel, and draw y based on f:

In [3]:
f = sim.sim_f(X)
y = sim.poisson_draw(f)
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=y, mode = 'markers', marker=dict(size = 2,))])

## Inference

We're interested in the following: given y and x, can we infer the function values f(x)? We've made an assumption with the Poisson likelihood here, but this implementation should work with any differentiable likelihood.

Construct a KroneckerSolver object (given a kernel), and run the inference. This should converge within a few Newton iterations.

In [4]:
ks = KroneckerSolver(GPy.kern.RBF(input_dim=1, variance=1.0, lengthscale=5.0), PoissonLike(), X, y, 0.5)

pred = ks.run(np.log(np.mean(y)), 10, f)

Iteration:  <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0>
 psi:  tf.Tensor(-1.09026e+06, shape=(), dtype=float32)
step 0.03125
Iteration:  tf.Tensor(1, shape=(), dtype=int32)
 psi:  tf.Tensor(-1.16807e+06, shape=(), dtype=float32)
step 0.0078125
Iteration:  tf.Tensor(2, shape=(), dtype=int32)
 psi:  tf.Tensor(-1.16822e+06, shape=(), dtype=float32)
step 0.0


Plot the inferred function values

In [5]:
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=np.array(pred[-2]), mode = 'markers', marker=dict(size = 2,))])

Next step: we're cheating above, since we don't optimize over kernel hyperparameters. Let's implement efficient optimization of the marginal likelihood over kernel hyperparameters.

Step after next: The limitation above (with Kronecker methods out of the box) is that the data needs to lie on a rectilinear grid. I'm currently working on implementing (both with my Kronecker implementation and with Edward) inducing point methods that relax this assumption.